<a href="https://colab.research.google.com/github/vladgap/LLC/blob/main/Chitin_LLC_simulation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing

In [2]:
!pip install MLNN
from MLNN import*

In [3]:
!wget -O LLC.py "https://raw.githubusercontent.com/vladgap/LLC/main/LLC_versions/LLC_4_4.py"
import LLC
from LLC import*

--2023-12-13 06:17:09--  https://raw.githubusercontent.com/vladgap/LLC/main/LLC_versions/LLC_4_4.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21856 (21K) [text/plain]
Saving to: ‘LLC.py’

LLC.py              100%[===================>]  21.34K  --.-KB/s    in 0.005s  

2023-12-13 06:17:09 (4.28 MB/s) - ‘LLC.py’ saved [21856/21856]



# EQUIL data

In [5]:
# Real solution R.T.
class EQUIL_chitin_1: # Br- only, no solvents dissolution
# gets xout [%solvent_aq, %Br_aq]
# calc xout , yout [%water_org, %Br_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %Br], [%solv, %Br], [%solv, %Br], ...]
        y=np.zeros_like(x)

        y[:,0]=3.6 # constant water

        # 1/D = A*aq+B
        A, B = -0.2121, 11.89
        # y = x*D = x/(1/D)
        y[:,1] = abs(x[:,1]/(A*x[:,1]+B)) # %Br_org

        self.xout = x # [[%solvent_aq, %Br_aq], ...]
        self.yout = y # [[%water_org, %Br_org], ...]

In [6]:
# Pilot solution-solvent 29-0-TCI
class EQUIL_chitin_2: # Br- only, no solvents dissolution
# gets xout [%solvent_aq, %Br_aq]
# calc xout , yout [%water_org, %Br_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %Br], [%solv, %Br], [%solv, %Br], ...]
        y=np.zeros_like(x)

        y[:,0]=3.6 # constant water

        # 1/D = A*aq+B
        A, B = -0.3894, 17.47
        # y = x*D = x/(1/D)
        y[:,1] = abs(x[:,1]/(A*x[:,1]+B)) # %Br_org

        self.xout = x # [[%solvent_aq, %Br_aq], ...]
        self.yout = y # [[%water_org, %Br_org], ...]

כאשר עקום ש"מ עבור מרכיב מסוים מתחיל מראשית הצירים (בד"כ עבור מרכיב עיקרי, שהוא לא מזהם), כדאי לעבוד בסימולטור עם מקדם חלוקה: לחשב ריכוז בקלה ע"י חילוק ריכוז בכבדה במקדם חלוקה. קורלציה ליניארית בודדת של מקדם חלוקה מבטיחה התכנסות לראשית הצירים. אין צורך בקורלציה מורכבת.

במקרה של התכנסות ריכוז מרכיב בקלה לערך מסוים גדול מאפס, לא ניתן להשתמש במקדם חלוקה, כי עבור ריכוזים נמוכים בכבדה מתקיים:
* מקדם חלוקה שואף ל-0
* אם מקרבים מקדם חלוקה לקו ישר, אז באותו תחום ערך בקלה יהיה קבוע במקום יורד מונוטונית
* שיפוע שהקירוב צריך להיות (ערך ההתכנסות)/1
לא נראה ששווה להתעסק עם זה. אם כבר מחלקים את הקורלציה לחלקים ואין דרישה של התכנסות לראשית הצירים, אז כדאי למצוא קורלציה של ריכוז בקלה מול בכבדה

In [36]:
# Extracton and washing
# שרית שלחה נתוני ש"מ שטיפה, שם הממס נשטף לאזור 0.4%
# הוחלט שברום לא נשטף מהממס מתחת ל-0.4%.
# היה נדרש לבנות מודל ש"מ עם התכנסות לאזור 0.4% בקלה בהגעה ל-0% בכבדה.

class EQUIL_chitin_3: # Br- only, no solvents dissolution
# gets xout [%solvent_aq, %Br_aq]
# calc xout , yout [%water_org, %Br_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %Br], [%solv, %Br], [%solv, %Br], ...]
        y=np.zeros_like(x)

        y[:,0]=3.6 # constant water

        # org=A+B*x^C
        A,B,C = 0.362401159,	1.88E-03,	2.323951931
        if np.any(x[:,1]<0):
          x[:,1]=x[:,1]*0
        y[:,1] = abs(A+B*abs(x[:,1])**C) # %Br_org

        self.xout = x # [[%solvent_aq, %Br_aq], ...]
        self.yout = y # [[%water_org, %Br_org], ...]

# Cases

## Extraction 1 -- eff 15%

התאמת כבדה מיצוי ל-29%

In [37]:
EQUIL_chitin_1([[0,32.4]]).yout

array([[3.6       , 6.45680715]])

In [38]:
test=LLC.Stage(5,1,[0,0],[0,39],EQUIL=EQUIL_chitin_1, convergence=1e-5)
test.output()

Oout= 5.432697636280635
yout= [3.6        4.36468847]
yout_tag= [3.6        4.36468847]  %water_org_equilib, %solubles_equilib
Aout= 0.5673023637193648
xout= [ 0.         26.94853146]
equilibrium efficiency= [1. 1.]
error= [9.06868516e-07 7.03202687e-08]
convergence= [-2.63287465e-08 -1.63436814e-06]


In [39]:
bat1_1=LLC.Battery(stages_num=5, Oin=7.895/1.806, Ain=1, yin=[2.4,0], xin=[0,39], eff=0.15, EQUIL= EQUIL_chitin_1)
bat1_1.runs

15

In [40]:
table1_1=LLC.BatteryTable(bat1_1).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
"Org in, ton/hr",4.371539,4.459498,4.490589,4.524178,4.561591
"y1 in, %",2.400000,3.600000,3.600000,3.600000,3.600000
"y2 in, %",0.000000,0.725099,1.387569,2.092990,2.866474
"Aq in, ton/hr",0.854685,0.885744,0.919313,0.956693,1.000000
"x1 in, %",0.000000,0.000000,0.000000,0.000000,0.000000
"x2 in, %",29.240452,31.595736,33.962143,36.401748,39.000000
"Org out, ton/hr",4.459495,4.490578,4.524169,4.561567,4.604901
"y1 out, %",3.600000,3.600000,3.600000,3.600000,3.600000
"y2 out, %",0.725027,1.387339,2.092807,2.865984,3.746152
"Aq out, ton/hr",0.766730,0.854664,0.885733,0.919304,0.956690


In [41]:
bat=bat1_1
fig1_1 = go.Figure()
n=1 #Br
fig1_1.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig1_1.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig1_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.1f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='top center', name='Operation', marker_color='green'))
fig1_1.update_xaxes(title_text="x, %")#, range=[0,40])
fig1_1.update_yaxes(title_text="y, %")#, range=[0,10])
# fig1_1.update_layout(title='', width=1200, height=600,)
fig1_1.show()

## Extraction 2 -- eff 100%

התאמת קלה מיצוי ל-7%

In [42]:
bat1_2=LLC.Battery(stages_num=5, Oin=7.895/1.806, Ain=1, yin=[2.4,0], xin=[0,39], eff=1, EQUIL= EQUIL_chitin_1)
bat1_2.runs

35

In [43]:
table1_2=LLC.BatteryTable(bat1_2).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
"Org in, ton/hr",4.371539,4.488848,4.537091,4.587489,4.650578
"y1 in, %",2.400000,3.600000,3.600000,3.600000,3.600000
"y2 in, %",0.000000,1.350638,2.361343,3.394846,4.656992
"Aq in, ton/hr",0.738812,0.787027,0.837424,0.900483,1.000000
"x1 in, %",0.000000,0.000000,0.000000,0.000000,0.000000
"x2 in, %",18.707094,23.467222,27.856467,32.656437,39.000000
"Org out, ton/hr",4.488848,4.537077,4.587489,4.650560,4.750094
"y1 out, %",3.600000,3.600000,3.600000,3.600000,3.600000
"y2 out, %",1.350638,2.361057,3.394846,4.656652,6.579224
"Aq out, ton/hr",0.621503,0.738798,0.787027,0.837412,0.900483


In [44]:
# Extraction efficiency
bat=bat1_2
eff=bat.Oout*bat.yout[1]/(bat.Ain*bat.xin[1])
print ('Extraction efficiency = {:.1%}'.format(eff))

Extraction efficiency = 80.1%


In [45]:
bat=bat1_2
fig1_2 = go.Figure()
n=1 #Br
fig1_2.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig1_2.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig1_2.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.1f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='top center', name='Operation', marker_color='green'))
fig1_2.update_xaxes(title_text="x, %")#, range=[0,40])
fig1_2.update_yaxes(title_text="y, %")#, range=[0,10])
# fig1_2.update_layout(title='', width=1200, height=600,)
fig1_2.show()

### Extraction efficiency

In [46]:
bat1_3=LLC.Battery(stages_num=5, Oin=8, Ain=1, yin=[2.4,0], xin=[0,39], eff=1, EQUIL= EQUIL_chitin_1)
bat1_3.runs

43

In [47]:
table1_3=LLC.BatteryTable(bat1_3).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
"Org in, ton/hr",8.000000,8.132808,8.169533,8.221544,8.306235
"y1 in, %",2.400000,3.600000,3.600000,3.600000,3.600000
"y2 in, %",0.000000,0.393972,0.825765,1.430418,2.398531
"Aq in, ton/hr",0.651929,0.688641,0.740652,0.825326,1.000000
"x1 in, %",0.000000,0.000000,0.000000,0.000000,0.000000
"x2 in, %",8.355011,13.048777,18.902364,26.851725,39.000000
"Org out, ton/hr",8.132808,8.169525,8.221544,8.306224,8.480909
"y1 out, %",3.600000,3.600000,3.600000,3.600000,3.600000
"y2 out, %",0.393972,0.825677,1.430418,2.398409,4.334594
"Aq out, ton/hr",0.519121,0.651923,0.688641,0.740646,0.825326


In [48]:
# Extraction efficiency
bat=bat1_3
eff=bat.Oout*bat.yout[1]/(bat.Ain*bat.xin[1])
print ('Extraction efficiency = {:.1%}'.format(eff))

Extraction efficiency = 94.3%


In [49]:
bat=bat1_3
fig1_3 = go.Figure()
n=1 #Br
fig1_3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig1_3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig1_3.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.1f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='top center', name='Operation', marker_color='green'))
fig1_3.update_xaxes(title_text="x, %")#, range=[0,40])
fig1_3.update_yaxes(title_text="y, %")#, range=[0,10])
# fig1_3.update_layout(title='', width=1200, height=600,)
fig1_3.show()

### Table

In [50]:
# stages=[5,6,7,8,9,10]
# ratios=[5,6,8,10,12]

# row=[]
# eff_table=[]
# for stage in stages:
#   for ratio in ratios:
#     bat=LLC.Battery(stages_num=stage, Oin=ratio, Ain=1, yin=[2.4,0], xin=[0,39], eff=1, max_iter=300, EQUIL= EQUIL_chitin_1)
#     eff=(bat.Oout*bat.yout[1]-bat.Oin*bat.yin[1])/(bat.Ain*bat.xin[1])
#     row.append(eff)
#   eff_table.append(row)
#   row=[]
# eff_table=pd.DataFrame(eff_table, columns=ratios, index=stages)
# print ('rows -- number of stages, columns -- ratio')
# eff_table

### Stack for vizualization

In [51]:
# stack=pd.DataFrame(eff_table.stack()).reset_index()
# stack.columns=['stages', 'ratio', 'eff']
# stack

In [52]:
# fig = go.Figure()
# fig.add_trace(go.Scatter3d(x=stack.stages, y=stack.ratio, z=stack.eff, mode='markers'))
# fig.update_layout(
#         scene = {
#             "xaxis": {"title": 'stages'},
#             "yaxis": {"title": 'ratio'},
#             "zaxis": {"title": 'efficiency'},
#             'camera_eye': {"x": -1.5, "y": -1.5, "z": 1.5},
#             # "aspectratio": {"x": 1, "y": 1, "z": 0.2}
#         })
# fig.show()

### test

In [53]:
bat1_3=LLC.Battery(stages_num=10, Oin=12, Ain=1, yin=[2.4,0], xin=[0,39], eff=1, max_iter=200, EQUIL= EQUIL_chitin_1)
bat1_3.runs

123

In [54]:
# Extraction efficiency
bat=bat1_3
eff=bat.Oout*bat.yout[1]/(bat.Ain*bat.xin[1])
print ('Extraction efficiency = {:.1%}'.format(eff))

Extraction efficiency = 99.9%


## Extraction 3 -- eff 100%

עקומת ש"מ נוספת אחרי הפעלת מערכת שולחנית

EQUIL_chitin_2

In [55]:
# stages=[5,6,7,8,9,10]
# ratios=[5,6,8,10,12]

# row=[]
# eff_table=[]
# for stage in stages:
#   for ratio in ratios:
#     bat=LLC.Battery(stages_num=stage, Oin=ratio, Ain=1, yin=[2.4,0], xin=[0,39], eff=1, max_iter=300, EQUIL= EQUIL_chitin_2)
#     eff=(bat.Oout*bat.yout[1]-bat.Oin*bat.yin[1])/(bat.Ain*bat.xin[1])
#     row.append(eff)
#   eff_table.append(row)
#   row=[]
# eff_table=pd.DataFrame(eff_table, columns=ratios, index=stages)
# print ('rows -- number of stages, columns -- ratio')
# eff_table

In [56]:
bat3_1=LLC.Battery(stages_num=5, Oin=7.895/1.806, Ain=1, yin=[2.4,0], xin=[0,39], eff=1, EQUIL= EQUIL_chitin_2)
bat3_1.runs

31

In [57]:
table3_1=LLC.BatteryTable(bat3_1).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
"Org in, ton/hr",4.371539,4.484966,4.524236,4.564982,4.620443
"y1 in, %",2.400000,3.600000,3.600000,3.600000,3.600000
"y2 in, %",0.000000,1.268323,2.094419,2.936186,4.058612
"Aq in, ton/hr",0.752878,0.792115,0.832861,0.888287,1.000000
"x1 in, %",0.000000,0.000000,0.000000,0.000000,0.000000
"x2 in, %",20.153213,23.932234,27.477641,31.781276,39.000000
"Org out, ton/hr",4.484966,4.524220,4.564982,4.620422,4.732156
"y1 out, %",3.600000,3.600000,3.600000,3.600000,3.600000
"y2 out, %",1.268323,2.094085,2.936186,4.058188,6.238508
"Aq out, ton/hr",0.639452,0.752861,0.792115,0.832848,0.888287


In [58]:
# Extraction efficiency
bat=bat3_1
eff=bat.Oout*bat.yout[1]/(bat.Ain*bat.xin[1])
print ('Extraction efficiency = {:.1%}'.format(eff))

Extraction efficiency = 75.7%


In [59]:
bat=bat3_1
fig3_1 = go.Figure()
n=1 #Br
fig3_1.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig3_1.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig3_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.1f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='top center', name='Operation', marker_color='green'))
fig3_1.update_xaxes(title_text="x, %")#, range=[0,40])
fig3_1.update_yaxes(title_text="y, %")#, range=[0,10])
# fig3_1.update_layout(title='', width=1200, height=600,)
fig3_1.show()

## Extraction 4 -- eff 100%

עקומות מיצוי ושטיפה ביחד. התכנסות ל-0.4% ברום

הזנת ממס שטוף עם 0.4% ברום

EQUIL_chitin_3

In [68]:
# stages=[5,6,7,8,9,10]
# ratios=[5,6,8,10,12]

# row=[]
# table=[]
# for stage in stages:
#   for ratio in ratios:
#     bat=LLC.Battery(stages_num=stage, Oin=ratio, Ain=1, yin=[2.4,0.4], xin=[0,39], eff=1, max_iter=300, EQUIL= EQUIL_chitin_3)#, convergence=1E-6)
#     eff=(bat.Oout*bat.yout[1]-bat.Oin*bat.yin[1])/(bat.Ain*bat.xin[1])
#     runs=bat.runs
#     row.append([eff,runs])
#   table.append(row)
#   row=[]
# table=np.array(table)
# eff_table=pd.DataFrame(table[:,:,0], columns=ratios, index=stages)
# print ('rows -- number of stages, columns -- ratio')
# eff_table

In [67]:
# run_table=pd.DataFrame(table[:,:,1], columns=ratios, index=stages)
# print ('rows -- number of stages, columns -- ratio')
# run_table

In [62]:
bat4_1=LLC.Battery(stages_num=5, Oin=5, Ain=1, yin=[2.4,0.4], xin=[0,39], eff=1, EQUIL= EQUIL_chitin_3)
bat=bat4_1
eff=(bat.Oout*bat.yout[1]-bat.Oin*bat.yin[1])/(bat.Ain*bat.xin[1])
print ('Extraction efficiency = {:.1%}'.format(eff))
bat.runs

Extraction efficiency = 77.5%


37

In [63]:
table4_1=LLC.BatteryTable(bat4_1).data

,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
"Org in, ton/hr",5.000000,5.107286,5.143272,5.189157,5.257692
"y1 in, %",2.400000,3.600000,3.600000,3.600000,3.600000
"y2 in, %",0.400000,1.241830,1.907774,2.744107,3.965048
"Aq in, ton/hr",0.731758,0.767715,0.813600,0.882097,1.000000
"x1 in, %",0.000000,0.000000,0.000000,0.000000,0.000000
"x2 in, %",17.958716,21.632637,25.849319,31.327744,39.000000
"Org out, ton/hr",5.107286,5.143257,5.189157,5.257669,5.375595
"y1 out, %",3.600000,3.600000,3.600000,3.600000,3.600000
"y2 out, %",1.241830,1.907488,2.744107,3.964637,5.993916
"Aq out, ton/hr",0.624472,0.731744,0.767715,0.813586,0.882097


In [64]:
bat=bat4_1
fig = go.Figure()
n=1 #Br
fig.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.1f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='top center', name='Operation', marker_color='green'))
fig.update_xaxes(title_text="x, %")#, range=[0,40])
fig.update_yaxes(title_text="y, %")#, range=[0,10])
# fig.update_layout(title='', width=1200, height=600,)
fig.show()